In [1]:
import gurobipy
from gurobipy import GRB

# Pyomo Solver

In [11]:
import numpy as np
from pyomo.environ import (
    ConcreteModel, Var, Objective, ConstraintList, SolverFactory, NonNegativeReals, Binary, maximize
)

# Step 1: Define problem parameters
V_H = 10  # Number of nodes (links)
E_H = 5   # Number of hyperedges

# Signal strengths, noise power, and interference matrix
N = np.random.rand(V_H)  # Noise power
I = np.random.rand(V_H, V_H)  # Interference matrix (I_ij)

# Hyperedges (list of node indices per hyperedge)
hyperedges = [
    [0, 1, 2],
    [2, 3, 4],
    [4, 5, 6],
    [6, 7, 8],
    [8, 9]
]

# Thresholds for hyperedges
theta = np.random.rand(E_H)

# Step 2: Create Pyomo model
model = ConcreteModel()

# Step 3: Define binary decision variables for each link (b_i)
model.b = Var(range(V_H), domain=Binary)  # b[i] = 1 if link i is active, 0 otherwise

# Step 4: Define the objective function (maximize throughput)
def throughput(model):
    total_throughput = 0
    for i in range(V_H):
        interference = sum(model.b[j] * I[i, j] for j in range(V_H) if j != i)  # Interference from other active links
        denominator = N[i] + interference  # Noise + interference
        total_throughput += (I[i,i] * model.b[i]) / denominator  # Contribution of link i to throughput
    return total_throughput

model.obj = Objective(rule=throughput, sense=maximize)

# Step 5: Add constraints for hyperedges (each hyperedge has a threshold)
model.constraints = ConstraintList()

for e_idx, hyperedge in enumerate(hyperedges):
    model.constraints.add(
        sum(model.b[i] for i in hyperedge) <= len(hyperedge)-1
    )

# Step 6: Solve the model
solver = SolverFactory('ipopt')  # Use Ipopt for nonlinear problems
result = solver.solve(model, tee=True)

# Step 7: Extract the results
optimal_decisions = [model.b[i].value for i in range(V_H)]

# Output the optimal link schedule
print("Optimal link schedule:", optimal_decisions)
print("Maximum throughput:", model.obj())


Ipopt 3.14.16: 
==> Warning: Treating 0 binary and 10 integer variables as continuous.


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       14
Number of nonzeros in Lagrangian Hessian.............:       55

Total number of variables............................:       10
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       10
                     variables with only upper bounds:        0
Tota

# Exhaustive Search Solver

In [67]:
# exhaustive search
def exh_solver(V_H, E_H, N, I, hyperedges):
    best_throughput = 0
    best_schedule = None

    for i in range(2**V_H):
        schedule = [int(x) for x in bin(i)[2:].zfill(V_H)]
        throughput = 0

        # check if the schedule satisfies the hyperedge constraints
        valid_schedule = True
        for hyperedge in hyperedges:
            if sum(schedule[j] for j in hyperedge) == len(hyperedge):
                valid_schedule = False
                break
        
        if not valid_schedule:
            continue

        # calculate throughput
        for i in range(V_H):
            interference = sum(schedule[j] * I[i, j] for j in range(V_H) if j != i)
            denominator = N[i] + interference
            throughput += (I[i,i] * schedule[i]) / denominator
        
        if throughput > best_throughput:
            best_throughput = throughput
            best_schedule = schedule

    return best_throughput, best_schedule
        

In [90]:
best_throughput, best_schedule = exh_solver(V_H, E_H, V_H*[0.01], I, hyperedges)

print("Exhaustive search results:")
print("Optimal link schedule:", best_schedule)
print("Maximum throughput:", best_throughput)

Exhaustive search results:
Optimal link schedule: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Maximum throughput: 99.63409923285451


# Learning-Based

# Data Preparation

Format of Data Should be as follows
Dictionary Keys: "$n$": Number of hypernodes ($|\mathcal{V}_H|$), "$E$": dictionray of hyperedges, "$I \in R^{n\times n}$": Interference and Power matrix, "$H \in \{0,1\}^{n\times m}$: Incident matrix ($m$ is the size of hyperedges) 

In [98]:
import torch
def hypergraph_generation(V_H, I, hyperedges):
    hypergraph = {}

    hypergraph["n"] = V_H
    hypergraph["E"] = {}

    for e_idx, hyperedge in enumerate(hyperedges):
        hypergraph["E"][e_idx] = hyperedge


    hypergraph["I"] = I

    # incident matrix H
    H = np.zeros((V_H, len(hyperedges)))
    for e_idx, hyperedge in enumerate(hyperedges):
        for node in hyperedge:
            H[node, e_idx] = 1

    hypergraph["H"] = torch.FloatTensor(H)

    
    return hypergraph
    


In [111]:
hyp = hypergraph_generation(V_H, I, hyperedges)

# Training

In [206]:
import importlib, model, networks, utils
importlib.reload(networks)
importlib.reload(utils)

importlib.reload(model)

<module 'model' from '/Users/rezaramezanpour/Documents/link_scheduling_hypergraph/model.py'>

<span style="color:red">**HyperGCN**</span> is defined in <span style="color:cyan">**networks**</span> file, but it's core layer <span style="color:red">*HyperGraphConvolution*</span> is defined in <span style="color:cyan">**utils**</span>.<br /> If you want change the entire model you can put your model in <span style="color:cyan">**networks**</span> file.<br />
The training process is defined in the <span style="color:cyan">**model**</span> file, so if you defined a new model and want to use it in training process, please change that file.<br /> Also the <span style="color:red">**Gumbel+LinSatNet**</span> is defined in <span style="color:cyan">**utils**</span> but it is used in training process. Our custom loss function (negative of our objecive) is defined in <span style="color:cyan">**model**</span>, i double checked it with random inputs, so it should be fine.

In [207]:
import model
HyperGCN = model.initialise(hyp)


In [ ]:
HyperGCN = model.train(HyperGCN, hyp, 1000)
